## This notebook contains the code for Boruta gene selection analysis
#### if u want to know more about boruta please follow this link https://www.researchgate.net/publication/220443685_Boruta_-_A_System_for_Feature_Selection

## Libraries Required

In [1]:
import warnings
import scipy as sp
import numpy as np
import pandas as pd
from sklearn.metrics import *
import matplotlib.pyplot as plt
from scipy.stats import randint
from __future__ import print_function, division
from sklearn.preprocessing import label_binarize
from sklearn.exceptions import ConvergenceWarning
from sklearn.multiclass import OneVsRestClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV
from sklearn.utils import check_random_state, check_X_y
from sklearn.base import TransformerMixin, BaseEstimator
warnings.filterwarnings("ignore", category=ConvergenceWarning)
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import RandomizedSearchCV
  # Assuming you are using the Boruta library


## Read the abundance Data

In [2]:
df = pd.read_csv('abundance.csv')

## This is how our data looks like

In [3]:
df

,Unnamed: 0,IGIB113083612,IGIB113002257842,IGIB113003405741,IGIB1130388376,IGIB1130493563,IGIB1130490942,IGIB113002277314,IGIB113002356641,IGIB113002374331,...,IGIB1130259871,IGIB113002393366,IGIB1130630558,IGIB1130927408,IGIB1130349380,IGIB113002297804,IGIB113002307585,IGIB113002393520,IGIB1130489912,IGIB113002190929
0,ENSG00000000419,6.467632,6.076438,5.683727,6.811358,6.247092,6.142734,6.689794,6.255150,6.708819,...,6.694202,7.901524,8.012131,6.277041,6.726043,6.754484,8.478701,7.480954,6.927056,7.173671
1,ENSG00000000457,5.623925,5.899103,5.866506,6.343263,5.953088,5.690140,5.413108,6.388867,6.437203,...,3.029895,7.261276,5.189570,6.216381,5.836147,6.007279,5.841475,6.698566,5.896934,6.383314
2,ENSG00000000460,4.633771,4.973219,4.867171,4.249772,4.745624,4.380331,4.518587,4.519699,4.286728,...,5.168628,7.824012,3.008087,5.219074,4.615122,4.285362,5.749757,5.931519,4.402074,4.918354
3,ENSG00000000938,10.832606,9.355316,11.291813,10.348202,11.474831,11.143710,10.281925,10.995389,10.225057,...,9.422564,9.727594,9.801387,10.701707,9.642007,12.127709,6.416470,9.838593,10.872446,11.114855
4,ENSG00000000971,4.529797,4.356021,4.054426,5.764041,4.428250,4.646614,3.927307,4.865398,3.825975,...,7.193971,9.959385,7.415074,6.312506,7.009848,4.504181,8.779507,7.661630,5.086159,4.989686
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17505,ENSG00000291336,4.689758,3.655313,4.079810,4.829806,3.939064,4.250575,5.011823,4.597696,5.708802,...,4.903466,5.898477,5.375463,5.431596,5.504055,4.174714,3.671268,5.656668,3.909011,4.577528
17506,ENSG00000291338,3.668090,4.382862,3.785763,4.614524,3.184557,3.621223,4.018349,3.961234,3.973248,...,2.715071,5.704757,2.715071,4.734596,5.282772,4.183960,4.537756,4.345707,3.915302,4.391036
17507,ENSG00000292246,4.711424,5.189790,4.291236,6.526125,4.179311,4.530280,4.113246,5.064168,4.399511,...,6.574159,9.482777,6.643534,5.070346,6.874143,5.211896,8.538606,8.540747,4.955177,5.550013
17508,ENSG00000292256,4.504034,3.628397,4.819180,4.551845,4.398893,4.509124,4.989322,4.496130,4.428645,...,5.721463,5.531927,2.680458,2.680458,3.934241,4.933003,4.090932,5.767498,4.499607,4.982629


 ## Preprocessing step of the data so that we can make it compatible as an input for boruta

In [4]:
def preprocessing(df,cat,ref):

    """
    df ----> Gene Matrix, where columns are the sample ids and rows are the gene Id and its corresponding read counts.
    cat----> It contains the info of the samples whether it belongs to mild moderate or severe
    ref ----> reference gene data from gtf to gain the info of biotype.

    This function return 3 matrix {Protein coding, pseudogene, lncrna}
    """
    ref = pd.read_csv(ref)
    batch = pd.read_csv(cat)
    ref = ref[['Gene_id','Gene_biotype']]
    df = df.rename(columns = {'Unnamed: 0':'Gene_id'})
    df = ref.merge(df,on  = 'Gene_id', how = 'inner')
    protein_coding = df[df['Gene_biotype'] == 'protein_coding'].reset_index().drop(['index','Gene_biotype'],axis  =1)
    lncrna = df[df['Gene_biotype'] == 'lncRNA'].reset_index().drop(['index','Gene_biotype'],axis  =1)
    pseudogene = df[df['Gene_biotype'] == 'pseudogenes'].reset_index().drop(['index','Gene_biotype'],axis  =1)
    
    protein_coding = protein_coding.T
    col_pc = protein_coding.iloc[0,0:]
    protein_coding.columns = col_pc
    protein_coding = protein_coding.iloc[1:]
    protein_coding = protein_coding.reset_index().rename(columns = {'index':'Sample_ID'})
    protein_coding = batch.merge(protein_coding,on  = 'Sample_ID', how = 'inner')

    lncrna = lncrna.T
    col_ln = lncrna.iloc[0,0:]
    lncrna.columns = col_ln
    lncrna = lncrna.iloc[1:]
    lncrna = lncrna.reset_index().rename(columns = {'index':'Sample_ID'})
    lncrna = batch.merge(lncrna,on  = 'Sample_ID', how = 'inner')

    pseudogene = pseudogene.T
    col_ps = pseudogene.iloc[0,0:]
    pseudogene.columns = col_ps
    pseudogene = pseudogene.iloc[1:]
    pseudogene = pseudogene.reset_index().rename(columns = {'index':'Sample_ID'})
    pseudogene = batch.merge(pseudogene,on  = 'Sample_ID', how = 'inner')
    
    
    return protein_coding,lncrna,pseudogene

In [5]:
protein_coding,lncrna,pseudogene = preprocessing(df,'batch_category.csv','referencegenes.csv')

## Initiate Boruta Pipelinie

In [6]:
class BorutaPy(BaseEstimator, TransformerMixin):

    def __init__(self, estimator, n_estimators=1000, perc=100, alpha=0.05,
                 two_step=True, max_iter=100, random_state=None, verbose=0,
                 early_stopping=False, n_iter_no_change=20):
        self.estimator = estimator
        self.n_estimators = n_estimators
        self.perc = perc
        self.alpha = alpha
        self.two_step = two_step
        self.max_iter = max_iter
        self.random_state = random_state
        self.verbose = verbose
        self.early_stopping = early_stopping
        self.n_iter_no_change = n_iter_no_change
        self.__version__ = '0.3'
        self._is_lightgbm = 'lightgbm' in str(type(self.estimator))

    def fit(self, X, y):
       

        return self._fit(X, y)

    def transform(self, X, weak=False, return_df=False):
       

        return self._transform(X, weak, return_df)

    def fit_transform(self, X, y, weak=False, return_df=False):
      

        self._fit(X, y)
        return self._transform(X, weak, return_df)

    def _validate_pandas_input(self, arg):
        try:
            return arg.values
        except AttributeError:
            raise ValueError(
                "input needs to be a numpy array or pandas data frame."
            )

    def _fit(self, X, y):
        # check input params
        self._check_params(X, y)

        if not isinstance(X, np.ndarray):
            X = self._validate_pandas_input(X) 
        if not isinstance(y, np.ndarray):
            y = self._validate_pandas_input(y)

        self.random_state = check_random_state(self.random_state)
        
        early_stopping = False
        if self.early_stopping:
            if self.n_iter_no_change >= self.max_iter:
                if self.verbose > 0:
                    print(
                        f"n_iter_no_change is bigger or equal to max_iter"
                        f"({self.n_iter_no_change} >= {self.max_iter}), "
                        f"early stopping will not be used."
                    )
            else:
                early_stopping = True
        
        # setup variables for Boruta
        n_sample, n_feat = X.shape
        _iter = 1
        # early stopping vars
        _same_iters = 1
        _last_dec_reg = None
        # holds the decision about each feature:
        # 0  - default state = tentative in original code
        # 1  - accepted in original code
        # -1 - rejected in original code
        dec_reg = np.zeros(n_feat, dtype=int)
        # counts how many times a given feature was more important than
        # the best of the shadow features
        hit_reg = np.zeros(n_feat, dtype=int)
        # these record the history of the iterations
        imp_history = np.zeros(n_feat, dtype=float)
        sha_max_history = []

        # set n_estimators
        if self.n_estimators != 'auto':
            self.estimator.set_params(n_estimators=self.n_estimators)

        # main feature selection loop
        while np.any(dec_reg == 0) and _iter < self.max_iter:
            # find optimal number of trees and depth
            if self.n_estimators == 'auto':
                # number of features that aren't rejected
                not_rejected = np.where(dec_reg >= 0)[0].shape[0]
                n_tree = self._get_tree_num(not_rejected)
                self.estimator.set_params(n_estimators=n_tree)

            # make sure we start with a new tree in each iteration
            if self._is_lightgbm:
                self.estimator.set_params(random_state=self.random_state.randint(0, 10000))
            else:
                self.estimator.set_params(random_state=self.random_state)

            # add shadow attributes, shuffle them and train estimator, get imps
            cur_imp = self._add_shadows_get_imps(X, y, dec_reg)

            # get the threshold of shadow importances we will use for rejection
            imp_sha_max = np.percentile(cur_imp[1], self.perc)

            # record importance history
            sha_max_history.append(imp_sha_max)
            imp_history = np.vstack((imp_history, cur_imp[0]))

            # register which feature is more imp than the max of shadows
            hit_reg = self._assign_hits(hit_reg, cur_imp, imp_sha_max)

            # based on hit_reg we check if a feature is doing better than
            # expected by chance
            dec_reg = self._do_tests(dec_reg, hit_reg, _iter)

            # print out confirmed features
            if self.verbose > 0 and _iter < self.max_iter:
                self._print_results(dec_reg, _iter, 0)
            if _iter < self.max_iter:
                _iter += 1
                
            # early stopping
            if early_stopping:
                if _last_dec_reg is not None and (_last_dec_reg == dec_reg).all():
                    _same_iters += 1
                    if self.verbose > 0:
                        print(
                            f"Early stopping: {_same_iters} out "
                            f"of {self.n_iter_no_change}"
                        )
                else:
                    _same_iters = 1
                    _last_dec_reg = dec_reg.copy()
                if _same_iters > self.n_iter_no_change:
                    break

        # we automatically apply R package's rough fix for tentative ones
        confirmed = np.where(dec_reg == 1)[0]
        tentative = np.where(dec_reg == 0)[0]
        # ignore the first row of zeros
        tentative_median = np.median(imp_history[1:, tentative], axis=0)
        # which tentative to keep
        tentative_confirmed = np.where(tentative_median
                                       > np.median(sha_max_history))[0]
        tentative = tentative[tentative_confirmed]

        # basic result variables
        self.n_features_ = confirmed.shape[0]
        self.support_ = np.zeros(n_feat, dtype=bool)
        self.support_[confirmed] = 1
        self.support_weak_ = np.zeros(n_feat, dtype=bool)
        self.support_weak_[tentative] = 1

        # ranking, confirmed variables are rank 1
        self.ranking_ = np.ones(n_feat, dtype=int)
        # tentative variables are rank 2
        self.ranking_[tentative] = 2
        # selected = confirmed and tentative
        selected = np.hstack((confirmed, tentative))
        # all rejected features are sorted by importance history
        not_selected = np.setdiff1d(np.arange(n_feat), selected)
        # large importance values should rank higher = lower ranks -> *(-1)
        imp_history_rejected = imp_history[1:, not_selected] * -1

        # update rank for not_selected features
        if not_selected.shape[0] > 0:
                # calculate ranks in each iteration, then median of ranks across feats
                iter_ranks = self._nanrankdata(imp_history_rejected, axis=1)
                rank_medians = np.nanmedian(iter_ranks, axis=0)
                ranks = self._nanrankdata(rank_medians, axis=0)

                # set smallest rank to 3 if there are tentative feats
                if tentative.shape[0] > 0:
                    ranks = ranks - np.min(ranks) + 3
                else:
                    # and 2 otherwise
                    ranks = ranks - np.min(ranks) + 2
                self.ranking_[not_selected] = ranks
        else:
            # all are selected, thus we set feature supports to True
            self.support_ = np.ones(n_feat, dtype=bool)

        self.importance_history_ = imp_history

        # notify user
        if self.verbose > 0:
            self._print_results(dec_reg, _iter, 1)
        return self

    def _transform(self, X, weak=False, return_df=False):
        # sanity check
        try:
            self.ranking_
        except AttributeError:
            raise ValueError('You need to call the fit(X, y) method first.')

        if weak:
            indices = self.support_ + self.support_weak_
        else:
            indices = self.support_

        if return_df:
            X = X.iloc[:, indices]
        else:
            X = X[:, indices]
        return X

    def _get_tree_num(self, n_feat):
        depth = None
        try:
            depth = self.estimator.get_params()['max_depth']
        except KeyError:
            warnings.warn(
                "The estimator does not have a max_depth property, as a result "
                " the number of trees to use cannot be estimated automatically."
            )
        if depth == None:
            depth = 10
        # how many times a feature should be considered on average
        f_repr = 100
        # n_feat * 2 because the training matrix is extended with n shadow features
        multi = ((n_feat * 2) / (np.sqrt(n_feat * 2) * depth))
        n_estimators = int(multi * f_repr)
        return n_estimators

    def _get_imp(self, X, y):
        try:
            self.estimator.fit(X, y)
        except Exception as e:
            raise ValueError('Please check your X and y variable. The provided '
                             'estimator cannot be fitted to your data.\n' + str(e))
        try:
            imp = self.estimator.feature_importances_
        except Exception:
            raise ValueError('Only methods with feature_importance_ attribute '
                             'are currently supported in BorutaPy.')
        return imp

    def _get_shuffle(self, seq):
        self.random_state.shuffle(seq)
        return seq

    def _add_shadows_get_imps(self, X, y, dec_reg):
        # find features that are tentative still
        x_cur_ind = np.where(dec_reg >= 0)[0]
        x_cur = np.copy(X[:, x_cur_ind])
        x_cur_w = x_cur.shape[1]
        # deep copy the matrix for the shadow matrix
        x_sha = np.copy(x_cur)
        # make sure there's at least 5 columns in the shadow matrix for
        while (x_sha.shape[1] < 5):
            x_sha = np.hstack((x_sha, x_sha))
        # shuffle xSha
        x_sha = np.apply_along_axis(self._get_shuffle, 0, x_sha)
        # get importance of the merged matrix
        imp = self._get_imp(np.hstack((x_cur, x_sha)), y)
        # separate importances of real and shadow features
        imp_sha = imp[x_cur_w:]
        imp_real = np.zeros(X.shape[1])
        imp_real[:] = np.nan
        imp_real[x_cur_ind] = imp[:x_cur_w]
        return imp_real, imp_sha

    def _assign_hits(self, hit_reg, cur_imp, imp_sha_max):
        # register hits for features that did better than the best of shadows
        cur_imp_no_nan = cur_imp[0]
        cur_imp_no_nan[np.isnan(cur_imp_no_nan)] = 0
        hits = np.where(cur_imp_no_nan > imp_sha_max)[0]
        hit_reg[hits] += 1
        return hit_reg

    def _do_tests(self, dec_reg, hit_reg, _iter):
        active_features = np.where(dec_reg >= 0)[0]
        hits = hit_reg[active_features]
        # get uncorrected p values based on hit_reg
        to_accept_ps = sp.stats.binom.sf(hits - 1, _iter, .5).flatten()
        to_reject_ps = sp.stats.binom.cdf(hits, _iter, .5).flatten()

        if self.two_step:
            # two step multicor process
            # first we correct for testing several features in each round using FDR
            to_accept = self._fdrcorrection(to_accept_ps, alpha=self.alpha)[0]
            to_reject = self._fdrcorrection(to_reject_ps, alpha=self.alpha)[0]

            # second we correct for testing the same feature over and over again
            # using bonferroni
            to_accept2 = to_accept_ps <= self.alpha / float(_iter)
            to_reject2 = to_reject_ps <= self.alpha / float(_iter)

            # combine the two multi corrections, and get indexes
            to_accept *= to_accept2
            to_reject *= to_reject2
        else:
            # as in th original Boruta, we simply do bonferroni correction
            # with the total n_feat in each iteration
            to_accept = to_accept_ps <= self.alpha / float(len(dec_reg))
            to_reject = to_reject_ps <= self.alpha / float(len(dec_reg))

        # find features which are 0 and have been rejected or accepted
        to_accept = np.where((dec_reg[active_features] == 0) * to_accept)[0]
        to_reject = np.where((dec_reg[active_features] == 0) * to_reject)[0]

        # updating dec_reg
        dec_reg[active_features[to_accept]] = 1
        dec_reg[active_features[to_reject]] = -1
        return dec_reg

    def _fdrcorrection(self, pvals, alpha=0.05):
       
        pvals = np.asarray(pvals)
        pvals_sortind = np.argsort(pvals)
        pvals_sorted = np.take(pvals, pvals_sortind)
        nobs = len(pvals_sorted)
        ecdffactor = np.arange(1, nobs + 1) / float(nobs)

        reject = pvals_sorted <= ecdffactor * alpha
        if reject.any():
            rejectmax = max(np.nonzero(reject)[0])
            reject[:rejectmax] = True

        pvals_corrected_raw = pvals_sorted / ecdffactor
        pvals_corrected = np.minimum.accumulate(pvals_corrected_raw[::-1])[::-1]
        pvals_corrected[pvals_corrected > 1] = 1
        # reorder p-values and rejection mask to original order of pvals
        pvals_corrected_ = np.empty_like(pvals_corrected)
        pvals_corrected_[pvals_sortind] = pvals_corrected
        reject_ = np.empty_like(reject)
        reject_[pvals_sortind] = reject
        return reject_, pvals_corrected_

    def _nanrankdata(self, X, axis=1):
        """
        Replaces bottleneck's nanrankdata with scipy and numpy alternative.
        """
        ranks = sp.stats.mstats.rankdata(X, axis=axis)
        ranks[np.isnan(X)] = np.nan
        return ranks

    def _check_params(self, X, y):
        """
        Check hyperparameters as well as X and y before proceeding with fit.
        """
        # check X and y are consistent len, X is Array and y is column
        X, y = check_X_y(X, y)
        if self.perc <= 0 or self.perc > 100:
            raise ValueError('The percentile should be between 0 and 100.')

        if self.alpha <= 0 or self.alpha > 1:
            raise ValueError('Alpha should be between 0 and 1.')

    def _print_results(self, dec_reg, _iter, flag):
        n_iter = str(_iter) + ' / ' + str(self.max_iter)
        n_confirmed = np.where(dec_reg == 1)[0].shape[0]
        n_rejected = np.where(dec_reg == -1)[0].shape[0]
        cols = ['Iteration: ', 'Confirmed: ', 'Tentative: ', 'Rejected: ']

        # still in feature selection
        if flag == 0:
            n_tentative = np.where(dec_reg == 0)[0].shape[0]
            content = map(str, [n_iter, n_confirmed, n_tentative, n_rejected])
            if self.verbose == 1:
                output = cols[0] + n_iter
            elif self.verbose > 1:
                output = '\n'.join([x[0] + '\t' + x[1] for x in zip(cols, content)])

        # Boruta finished running and tentatives have been filtered
        else:
            n_tentative = np.sum(self.support_weak_)
            n_rejected = np.sum(~(self.support_|self.support_weak_))
            content = map(str, [n_iter, n_confirmed, n_tentative, n_rejected])
            result = '\n'.join([x[0] + '\t' + x[1] for x in zip(cols, content)])
            output = "\n\nBorutaPy finished running.\n\n" + result
        print(output)

In [7]:
def HyperParamTuning(df):
    # Separate features and target
    feat = df.drop(['Sample_ID', 'category'], axis=1)
    target = df['category']
    target_numeric = target.astype('category').cat.codes
    
    # Create a RandomForestClassifier
    rf = RandomForestClassifier(class_weight='balanced', random_state=42)

    # Use RandomizedSearchCV for hyperparameter tuning
    param_dist = {
        'n_estimators': [50, 100, 150],
        'max_depth': [None, 10, 20, 30],
        'min_samples_split': [2, 5, 10],
        'min_samples_leaf': [1, 2, 4]
    }
    
    random_search = RandomizedSearchCV(
        rf, param_distributions=param_dist, n_iter=10, cv=5, scoring='accuracy', random_state=42
    )
    
    print('started fitting random search')
    random_search.fit(feat, target_numeric)

    # Get the best estimator from the search
    best_rf = random_search.best_estimator_
    print('Done fitting random search')
   
    return best_rf,feat,target_numeric



In [8]:
best_rf_pc,feat_pc,target_numeric_pc = HyperParamTuning(protein_coding)


started fitting random search
Done fitting random search


In [9]:
best_rf_ln,feat_ln,target_numeric_ln = HyperParamTuning(lncrna)


started fitting random search
Done fitting random search


In [ ]:
best_rf_ps,feat_ps,target_numeric_ps = HyperParamTuning(pseudogene)


In [ ]:
def GeneSelection(model,feat,target):
    boruta_selector = BorutaPy(model, n_estimators='auto', verbose=2, random_state=42,max_iter = 1000)
    boruta_selector.fit(feat.values, target.values)
    selected_features = feat.columns[boruta_selector.support_].tolist()

    return selected_features


In [13]:
selected_features_pc = GeneSelection(best_rf_pc,feat_pc,target_numeric_pc)

Iteration: 	1 / 1000
Confirmed: 	0
Tentative: 	13374
Rejected: 	0
Iteration: 	2 / 1000
Confirmed: 	0
Tentative: 	13374
Rejected: 	0
Iteration: 	3 / 1000
Confirmed: 	0
Tentative: 	13374
Rejected: 	0
Iteration: 	4 / 1000
Confirmed: 	0
Tentative: 	13374
Rejected: 	0
Iteration: 	5 / 1000
Confirmed: 	0
Tentative: 	13374
Rejected: 	0
Iteration: 	6 / 1000
Confirmed: 	0
Tentative: 	13374
Rejected: 	0
Iteration: 	7 / 1000
Confirmed: 	0
Tentative: 	13374
Rejected: 	0
Iteration: 	8 / 1000
Confirmed: 	0
Tentative: 	384
Rejected: 	12990
Iteration: 	9 / 1000
Confirmed: 	0
Tentative: 	384
Rejected: 	12990
Iteration: 	10 / 1000
Confirmed: 	10
Tentative: 	374
Rejected: 	12990
Iteration: 	11 / 1000
Confirmed: 	10
Tentative: 	374
Rejected: 	12990
Iteration: 	12 / 1000
Confirmed: 	10
Tentative: 	300
Rejected: 	13064
Iteration: 	13 / 1000
Confirmed: 	21
Tentative: 	289
Rejected: 	13064
Iteration: 	14 / 1000
Confirmed: 	21
Tentative: 	289
Rejected: 	13064
Iteration: 	15 / 1000
Confirmed: 	21
Tentative: 	289

In [14]:
selected_features_ln = GeneSelection(best_rf_ln,feat_ln,target_numeric_ln)

Iteration: 	1 / 1000
Confirmed: 	0
Tentative: 	3358
Rejected: 	0
Iteration: 	2 / 1000
Confirmed: 	0
Tentative: 	3358
Rejected: 	0
Iteration: 	3 / 1000
Confirmed: 	0
Tentative: 	3358
Rejected: 	0
Iteration: 	4 / 1000
Confirmed: 	0
Tentative: 	3358
Rejected: 	0
Iteration: 	5 / 1000
Confirmed: 	0
Tentative: 	3358
Rejected: 	0
Iteration: 	6 / 1000
Confirmed: 	0
Tentative: 	3358
Rejected: 	0
Iteration: 	7 / 1000
Confirmed: 	0
Tentative: 	3358
Rejected: 	0
Iteration: 	8 / 1000
Confirmed: 	0
Tentative: 	70
Rejected: 	3288
Iteration: 	9 / 1000
Confirmed: 	0
Tentative: 	70
Rejected: 	3288
Iteration: 	10 / 1000
Confirmed: 	0
Tentative: 	70
Rejected: 	3288
Iteration: 	11 / 1000
Confirmed: 	0
Tentative: 	70
Rejected: 	3288
Iteration: 	12 / 1000
Confirmed: 	0
Tentative: 	48
Rejected: 	3310
Iteration: 	13 / 1000
Confirmed: 	0
Tentative: 	48
Rejected: 	3310
Iteration: 	14 / 1000
Confirmed: 	0
Tentative: 	48
Rejected: 	3310
Iteration: 	15 / 1000
Confirmed: 	0
Tentative: 	48
Rejected: 	3310
Iteration: 

In [15]:
selected_features_ps = GeneSelection(best_rf_ps,feat_ps,target_numeric_ps)

Iteration: 	1 / 1000
Confirmed: 	0
Tentative: 	420
Rejected: 	0
Iteration: 	2 / 1000
Confirmed: 	0
Tentative: 	420
Rejected: 	0
Iteration: 	3 / 1000
Confirmed: 	0
Tentative: 	420
Rejected: 	0
Iteration: 	4 / 1000
Confirmed: 	0
Tentative: 	420
Rejected: 	0
Iteration: 	5 / 1000
Confirmed: 	0
Tentative: 	420
Rejected: 	0
Iteration: 	6 / 1000
Confirmed: 	0
Tentative: 	420
Rejected: 	0
Iteration: 	7 / 1000
Confirmed: 	0
Tentative: 	420
Rejected: 	0
Iteration: 	8 / 1000
Confirmed: 	0
Tentative: 	59
Rejected: 	361
Iteration: 	9 / 1000
Confirmed: 	0
Tentative: 	59
Rejected: 	361
Iteration: 	10 / 1000
Confirmed: 	0
Tentative: 	59
Rejected: 	361
Iteration: 	11 / 1000
Confirmed: 	0
Tentative: 	59
Rejected: 	361
Iteration: 	12 / 1000
Confirmed: 	0
Tentative: 	45
Rejected: 	375
Iteration: 	13 / 1000
Confirmed: 	2
Tentative: 	43
Rejected: 	375
Iteration: 	14 / 1000
Confirmed: 	2
Tentative: 	43
Rejected: 	375
Iteration: 	15 / 1000
Confirmed: 	2
Tentative: 	43
Rejected: 	375
Iteration: 	16 / 1000
Conf

## Protein Coding gene selected using boruta

In [20]:
selected_features_pc

['ENSG00000158711',
 'ENSG00000121481',
 'ENSG00000284770',
 'ENSG00000117395',
 'ENSG00000117593',
 'ENSG00000188290',
 'ENSG00000171824',
 'ENSG00000117266',
 'ENSG00000271425',
 'ENSG00000007923',
 'ENSG00000180667',
 'ENSG00000171793',
 'ENSG00000160818',
 'ENSG00000143952',
 'ENSG00000155744',
 'ENSG00000091409',
 'ENSG00000115762',
 'ENSG00000115457',
 'ENSG00000116106',
 'ENSG00000115760',
 'ENSG00000119787',
 'ENSG00000163655',
 'ENSG00000034533',
 'ENSG00000144713',
 'ENSG00000197548',
 'ENSG00000176945',
 'ENSG00000145241',
 'ENSG00000138764',
 'ENSG00000164104',
 'ENSG00000178988',
 'ENSG00000185477',
 'ENSG00000109133',
 'ENSG00000150712',
 'ENSG00000168944',
 'ENSG00000152942',
 'ENSG00000145741',
 'ENSG00000197451',
 'ENSG00000113048',
 'ENSG00000145996',
 'ENSG00000135334',
 'ENSG00000274641',
 'ENSG00000137312',
 'ENSG00000272325',
 'ENSG00000010810',
 'ENSG00000136231',
 'ENSG00000164713',
 'ENSG00000135164',
 'ENSG00000160993',
 'ENSG00000106591',
 'ENSG00000179144',


## Lncrna Selected using boruta

In [21]:
selected_features_ln

['ENSG00000228606',
 'ENSG00000272449',
 'ENSG00000272145',
 'ENSG00000290041',
 'ENSG00000225649',
 'ENSG00000288836',
 'ENSG00000291124',
 'ENSG00000290871',
 'ENSG00000226530',
 'ENSG00000289316',
 'ENSG00000291048',
 'ENSG00000224307',
 'ENSG00000221817',
 'ENSG00000256039',
 'ENSG00000247774',
 'ENSG00000235532',
 'ENSG00000289083',
 'ENSG00000275888',
 'ENSG00000277089',
 'ENSG00000268573',
 'ENSG00000291067',
 'ENSG00000288719']

## Pseduogene selected using boruta 

In [22]:
selected_features_ps

['ENSG00000234851',
 'ENSG00000230409',
 'ENSG00000132967',
 'ENSG00000246596',
 'ENSG00000217527',
 'ENSG00000218809',
 'ENSG00000183444',
 'ENSG00000165178',
 'ENSG00000236567',
 'ENSG00000244398',
 'ENSG00000258352',
 'ENSG00000240342',
 'ENSG00000232587',
 'ENSG00000223959',
 'ENSG00000214425']

In [25]:
protein_coding = PostProcessing(protein_coding,selected_features_pc)

In [28]:
protein_coding

,Sample_ID,category,ENSG00000158711,ENSG00000121481,ENSG00000284770,ENSG00000117395,ENSG00000117593,ENSG00000188290,ENSG00000171824,ENSG00000117266,...,ENSG00000125743,ENSG00000130255,ENSG00000186431,ENSG00000172081,ENSG00000067048,ENSG00000100284,ENSG00000128191,ENSG00000154734,ENSG00000183255,ENSG00000142192
0,IGIB113000849750,mild,7.383388,5.365834,4.346439,4.421764,3.722133,7.501281,5.716382,4.130014,...,5.731206,8.364853,9.72584,9.705569,8.223652,9.468817,5.261334,4.058145,9.532755,7.213095
1,IGIB113001484596,mild,5.864997,4.427129,5.189263,4.166348,4.630951,7.919929,5.761629,5.45363,...,5.440821,7.572712,10.737821,10.050547,8.565993,9.835887,5.833667,5.257737,9.179533,6.916128
2,IGIB113002156413,mild,7.786278,5.856009,4.763183,4.789456,4.004076,7.238069,6.386381,4.338986,...,6.238875,8.320083,9.426954,9.009914,9.096065,9.170082,4.782876,3.821914,9.008836,7.456846
3,IGIB113002224539,mild,6.577487,5.201215,4.815029,4.186434,3.956746,8.052248,5.298212,4.523816,...,5.757348,8.338068,10.805577,9.940364,3.322247,10.559174,5.705938,3.647087,9.854809,7.524495
4,IGIB113002257842,mild,7.866947,5.741448,4.264734,4.231073,5.174939,4.852259,4.97545,3.152815,...,5.701162,8.403633,10.242025,11.019502,4.483703,9.271521,4.967891,4.620415,9.000816,8.097238
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
107,IGIB1130581789,severe,7.185213,6.576621,5.295892,6.023768,4.585234,5.989932,7.410619,3.282507,...,7.382007,9.743063,9.419201,8.595733,9.759901,9.307612,5.217771,3.112073,9.183453,9.442274
108,IGIB1130630558,severe,6.593348,7.863512,6.924312,6.472316,7.128248,5.787384,6.694107,2.918514,...,7.349402,9.574144,8.624466,7.684762,9.976802,9.004018,6.189227,3.02625,9.156831,8.507146
109,IGIB1130925327,severe,6.994577,6.0972,4.621329,5.107753,4.685862,3.537951,7.284443,4.640069,...,7.6996,9.839856,9.104871,7.757686,10.708712,8.675682,5.800424,4.621882,9.050459,6.548125
110,IGIB1130927408,severe,6.087678,5.7225,5.172901,5.385487,4.979868,6.346178,6.702339,3.784988,...,6.61148,8.721974,9.869153,8.521139,3.374427,8.99342,6.614055,3.02625,8.412335,8.13854


In [26]:
lncrna = PostProcessing(lncrna,selected_features_ln)

In [29]:
lncrna

,Sample_ID,category,ENSG00000228606,ENSG00000272449,ENSG00000272145,ENSG00000290041,ENSG00000225649,ENSG00000288836,ENSG00000291124,ENSG00000290871,...,ENSG00000221817,ENSG00000256039,ENSG00000247774,ENSG00000235532,ENSG00000289083,ENSG00000275888,ENSG00000277089,ENSG00000268573,ENSG00000291067,ENSG00000288719
0,IGIB113000849750,mild,3.393696,5.176199,3.116306,4.736968,3.823076,3.989919,4.402871,7.081518,...,3.922318,3.366907,5.586906,3.342666,4.138615,6.739795,3.214431,3.846309,7.780436,3.903799
1,IGIB113001484596,mild,3.756077,6.227766,4.586185,5.410261,5.745809,4.102743,4.768421,7.075495,...,6.231755,5.016794,5.443252,4.634843,3.920713,6.558285,4.132329,3.814242,7.302165,3.580753
2,IGIB113002156413,mild,3.09661,4.364346,4.196103,3.631024,3.607765,4.252817,4.69512,6.744937,...,4.490518,3.789645,5.682987,3.158193,3.947941,5.158623,3.426264,3.523964,7.423283,3.759353
3,IGIB113002224539,mild,3.910101,6.109291,4.526445,4.043277,4.191049,3.881226,4.978112,6.000902,...,5.425319,3.776143,4.926535,3.473027,5.081995,6.444782,3.765713,4.308316,7.313628,4.321124
4,IGIB113002257842,mild,4.110453,7.016891,4.592232,6.528928,4.3661,2.913833,5.808553,6.968498,...,4.715154,4.318313,5.783158,2.570834,3.445897,7.765539,2.36657,3.341752,7.161449,3.807219
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
107,IGIB1130581789,severe,4.640736,4.336248,4.766741,3.871263,2.686007,5.552801,6.337066,8.155131,...,5.905209,6.253349,7.59608,3.903799,4.811756,5.831799,4.259842,4.839456,6.24166,3.800383
108,IGIB1130630558,severe,6.73094,2.854354,5.507822,2.564716,3.216121,2.686671,2.79161,8.306954,...,2.964446,5.163178,7.200411,2.968194,2.950556,7.548168,7.392611,5.528292,6.703615,5.556268
109,IGIB1130925327,severe,3.27409,5.442974,5.114919,3.733103,5.20076,4.473268,7.025807,6.315134,...,5.432468,7.061169,6.293421,5.374744,5.020138,3.502451,2.973952,4.143962,7.008267,4.37202
110,IGIB1130927408,severe,3.999571,4.540051,4.729027,3.778355,3.216121,5.535608,5.276017,8.249071,...,5.591428,5.652732,7.154189,4.196861,5.227971,6.657245,4.79379,5.011791,6.870311,5.612968


In [27]:
pseudogene = PostProcessing(pseudogene,selected_features_ps)

In [30]:
pseudogene

,Sample_ID,category,ENSG00000234851,ENSG00000230409,ENSG00000132967,ENSG00000246596,ENSG00000217527,ENSG00000218809,ENSG00000183444,ENSG00000165178,ENSG00000236567,ENSG00000244398,ENSG00000258352,ENSG00000240342,ENSG00000232587,ENSG00000223959,ENSG00000214425
0,IGIB113000849750,mild,7.767143,4.415993,6.376321,3.693547,4.013369,5.657341,3.90157,10.403825,4.007397,6.155713,3.711438,7.329952,4.461518,4.838655,5.195427
1,IGIB113001484596,mild,7.808965,3.788586,4.34909,4.063922,4.300458,4.744479,4.351585,9.251843,2.867436,7.525433,2.753481,6.350414,4.157192,4.505077,4.041947
2,IGIB113002156413,mild,7.982593,4.335493,5.679668,3.785966,3.701473,4.99343,4.219661,10.351828,3.928359,6.813189,3.243649,7.159495,4.587518,5.201479,5.665678
3,IGIB113002224539,mild,7.619297,4.669483,5.548975,3.532907,4.308827,4.66396,2.616095,9.820147,3.74556,6.601214,2.774072,6.431403,3.455814,4.824798,2.691084
4,IGIB113002257842,mild,8.221667,3.463011,5.428075,3.1445,5.11465,5.027389,4.303364,8.435416,5.821256,6.927659,2.55311,7.886205,4.946954,4.18887,4.971547
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
107,IGIB1130581789,severe,10.151376,4.866351,7.341479,4.85197,3.68286,4.315888,3.322302,8.71856,3.514838,8.983962,3.429047,6.857262,4.655313,5.750594,6.6748
108,IGIB1130630558,severe,9.047081,2.912609,2.643622,6.340482,2.696228,5.40824,3.137626,7.202627,2.672986,8.444658,2.71856,8.604207,2.861471,6.566482,2.729751
109,IGIB1130925327,severe,9.727064,2.912609,5.935803,4.529058,2.696228,2.644806,3.137626,7.145122,2.672986,6.303316,4.097497,8.57973,4.798286,6.305203,5.87925
110,IGIB1130927408,severe,8.55867,5.590852,7.261906,4.760709,2.696228,2.644806,3.137626,9.225709,2.672986,6.827294,3.835376,8.094254,4.548323,5.202253,5.738415


In [ ]:

def train_and_evaluate_model(best_rf,df,name, n_iter=10 ,test_size=0.15):
    X = df.drop(['Sample_ID', 'category'], axis=1)
    y = df['category']
    
    # Split the data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=42)
    best_rf.fit(X_train, y_train)
    y_test_bin = label_binarize(y_test, classes=best_rf.classes_)
    classifier = OneVsRestClassifier(best_rf)
    y_score = classifier.fit(X_train, y_train).predict_proba(X_test)
    y_pred = classifier.predict(X_test)
    
    # Make predictions on the test set
   
    
    # Calculate accuracy
    accuracy = accuracy_score(y_test, y_pred)
    
    
# Plot the SHAP stacked bar plot for all classes
    #shap.summary_plot(shap_values_stacked, X_test, feature_names=X.columns, plot_type="bar", class_names=["Mild", "Moderate", "Severe"])

    plt.show()


    fpr = dict()
    tpr = dict()
    roc_auc = dict()
    for i in range(len(best_rf.classes_)):
        fpr[i], tpr[i], _ = roc_curve(y_test_bin[:, i], y_score[:, i])
        roc_auc[i] = auc(fpr[i], tpr[i])

# Plot ROC curve for each class
    plt.figure(dpi = 600,figsize=(8, 5))
    colors = ['#68B984', '#FFC93C', '#6499E9']  # Add more colors if needed
    class_labels = best_rf.classes_

    for i, color, label in zip(range(len(class_labels)), colors, class_labels):
        plt.plot(fpr[i], tpr[i], color=color, lw=2, label=f'{label} (AUC = {roc_auc[i]:.2f})')

    plt.plot([0, 1], [0, 1], 'k--', lw=2)
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title(name)
    plt.legend(loc="lower right")
    #plt.savefig(name + '_auroc_plot.svg')
    plt.show()
    

    return X_test,best_rf



In [ ]:
X_test_pc,best_rf_model_pc = train_and_evaluate_model(best_rf_pc,protein_coding, 'Protein Coding',n_iter=5, test_size=0.20)

In [ ]:
X_test_ln,best_rf_model_ln = train_and_evaluate_model(best_rf_ln,lncrna, 'LncRNA',n_iter=5, test_size=0.20)

In [ ]:
X_test_ps,best_rf_model_ps = train_and_evaluate_model(best_rf_ps,pseudogene, 'PSeudogene',n_iter=5, test_size=0.20)

In [ ]:
X_test_all,best_rf_model_all = train_and_evaluate_model(best_rf_all,all_data_sel, 'All Data',n_iter=5, test_size=0.20)

In [ ]:
## Shap -- > A model interpretability so that we can know the exact impact 

In [ ]:
import shap
import seaborn as sns
from matplotlib.colors import ListedColormap
def shapleyplot(model, X_test_data, name):
    colors = ['#68B984', '#FFC93C', '#6499E9']
    cmap = ListedColormap(colors)
    
    explainer = shap.TreeExplainer(model)
    shap_values = explainer.shap_values(X_test_data)
    
    plt.figure(dpi=600, figsize=(15, 5))
    shap.summary_plot(shap_values, X_test_data, max_display=120, class_names=['mild', 'moderate', 'severe'], color=cmap, show=False)
    #plt.savefig(name + '_shapplot.svg')
    return shap_values


In [ ]:
shap_pseudo = shapleyplot(best_rf_model_all,X_test_all, 'All Data')

In [ ]:
shap_pc = shapleyplot(best_rf_model_pc,X_test_pc, 'Protein Coding')

In [ ]:
shap_ln = shapleyplot(best_rf_model_ln,X_test_ln, 'LncRNA')

In [ ]:
shap_ps = shapleyplot(best_rf_model_ps,X_test_ps, 'Pseudogene')